In [44]:
## Loading relevant packages
library(slackr)
library(AirSensor)
library(MazamaSpatialUtils)
library(lubridate)
library(dplyr)

## Loading Spatial data 
## More details on Air Sensor Git hub page if it's not setup already
initializeMazamaSpatialUtils("/home/ubuntu/Git/Sensor_Alert/Spatial_data")

## Settng the Country Code API link and look back days 
pas <- pas_createNew(countryCodes = "IN",
                     baseUrl = 'https://www.purpleair.com/json?all=true',lookbackDays = 30,
                     includePWFSL = F)
## Changing the time zone to Asia/Kolkata
pas$lastSeenDate <- with_tz(pas$lastSeenDate,tz="Asia/Kolkata")



In [45]:
Colocated_Sensors =read.csv("/home/ubuntu/Git/Sensor_Alert/Data/25_Colocated_Sensors_22Sep21.csv")
SAMOSA_pas = pas%>% pas_filter(!grepl("B",label),grepl("SAMOSA",label))
head(Colocated_Sensors)


,Dongles,Sensor_name,MacID,Status
,<fct>,<fct>,<fct>,<fct>
1,SAMOSA1,SAMOSA_0060,10:52:1c:42:1d:de,Done
2,SAMOSA1,SAMOSA_0061,F4:cf:a2:dd:42:d3,Done
3,SAMOSA1,SAMOSA_0063,F4:cf:a2:dd:42:4c,Done
4,SAMOSA1,SAMOSA_0065,50:2:91:f7:44:c4,Done
5,SAMOSA1,SAMOSA_0066,10:52:1c:41:88:33,Done
6,SAMOSA2,SAMOSA_0067,A4:cf:12:f9:ff:d5,Done


In [49]:
#Send notification to slack channel
time <- Sys.time()
## Changing the time zone to UTC
time <- with_tz(time,tz="Asia/Kolkata")
## Creating 5 minutes lag for the test (If any sensor is inactive for more than an hour we send an alert)
time_check = time-minutes(5)
## Finding any inactive sensors
Inactive_sensors = SAMOSA_pas %>%mutate(Sensor_name=label)%>% filter(label %in% Colocated_Sensors$Sensor_name) %>% 
filter(time_check>lastSeenDate) %>% left_join(Colocated_Sensors,by="Sensor_name") %>%
select(Sensor_name,lastSeenDate,Dongles)
head(Inactive_sensors)
dim(Inactive_sensors)[1]

Sensor_name,lastSeenDate,Dongles
<chr>,<dttm>,<fct>
SAMOSA_0060,2021-09-22 18:53:06,SAMOSA1
SAMOSA_0061,2021-09-22 18:53:15,SAMOSA1
SAMOSA_0063,2021-09-22 18:53:12,SAMOSA1
SAMOSA_0065,2021-09-22 18:53:08,SAMOSA1
SAMOSA_0066,2021-09-22 18:53:19,SAMOSA1
SAMOSA_0067,2021-09-22 18:53:31,SAMOSA2


[1] 25

In [50]:
## Adding the R environment file. This file contains the API used for analysis
readRenviron("/home/ubuntu/Git/API.Renviron")

slackr_setup(channel = "pa_sensor_alert",
             bot_user_oauth_token =  Sys.getenv('bot_user_oauth_token'),
             incoming_webhook_url = Sys.getenv('incoming_webhook_url'))

[1] "Successfully connected to Slack"

In [51]:
if (dim(Inactive_sensors)[1] > 0){
    
    slackr("Time of Query",time)
    slackr("Total Inactive sensors", dim(Inactive_sensors)[1])
    ## Sending the Slack alert if the sensors is inactive at any moment
    slackr("Name of Inactive sensors at the moment", pander::pandoc.table(Inactive_sensors))
}